In [8]:
 #!pip install stop_words
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from stop_words import get_stop_words
import os
from datetime import datetime
import pickle

In [9]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/MLProject_KenyaFinancial

# /content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial/ mypath

Mounted at /content/drive
[Errno 2] No such file or directory: '/content/drive/MyDrive/MLProject_KenyaFinancial'
/content


In [10]:
x_path = "/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial/clean_data/XoheImp.csv" 
y_path = "/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial/clean_data/Y.csv"

In [11]:
#Set X and y values
X = pd.read_csv(x_path)
y = pd.read_csv(y_path)

Recoding Y as a binary variable with mobile money = 1 and the rest = 0

In [30]:
from sklearn.preprocessing import label_binarize #one-vs-all scheme
yb = label_binarize(["b'Mobile money transfer (M-PESA, Airtel Money, Orange Money, Yu Cash, Tangaza, etc)'"], classes= y['trx_mode_desc'])


Starting with machine learning preprocessing using Ximp and Y

In [ ]:
# Import dependencies 
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

Tree

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

In [ ]:
XtrainVal, Xtest, YtrainVal, Ytest = train_test_split(X, Yd, test_size=0.2, random_state=42)
Xtrain, Xval, Ytrain, Yval = train_test_split(XtrainVal, YtrainVal, test_size=0.25, random_state=42)

In [ ]:
# Here we use the best model for the tree with a depth = 50, that had an MCC score of 92%
clf = DecisionTreeClassifier(random_state=42, max_depth=50)

In [ ]:
clf = clf.fit(Xtrain, Ytrain)

In [ ]:
YpredVal = clf.predict(Xval)
YpredTrain = clf.predict(Xtrain)

In [ ]:
# ROC AUC SCORE
